### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [1]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def extract_words(sent):
    #print("line tokenizing : \n{0}".format(sent))
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [4]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [5]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.72700226]], dtype=float32)

In [6]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("file being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [7]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

file being read : ../indata/eminem-not-affraid.csv
file being read : ../indata/got8.csv
file being read : ../indata/house-of-cards.csv
file being read : ../indata/samsung-vr.csv
file being read : ../indata/tonight-show.csv
file being read : ../indata/tswift.csv
file being read : ../indata/warriors.csv


In [8]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS
0,Give Him Some Water Man,2018-12-03T10:11:17.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[give, him, some, water, man]","[-0.084303185, 0.43828505, 0.24113432, -0.1528..."
1,What s going on Eminem,2018-12-03T10:09:18.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[what, s, going, on, eminem]","[0.14959222, -0.18436295, 0.3981964, 0.1262267..."
2,No one can beat you,2018-12-03T09:46:24.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[no, one, can, beat, you]","[-0.06286393, 0.46472645, 0.2018007, 0.1916378..."
3,EMINEM THE ALLTIME BEST RAPPER IN THE WORLD,2018-12-03T09:45:16.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[eminem, the, alltime, best, rapper, in, the, ...","[0.070282124, 0.07256028, 0.07732015, 0.011419..."
4,Much love from india,2018-12-03T09:03:50.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[much, love, from, india]","[0.16853374, 0.22485605, 0.123892896, 0.045539..."


In [9]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['eminem-not-affraid.csv' 'got8.csv' 'house-of-cards.csv' 'samsung-vr.csv'
 'tonight-show.csv' 'tswift.csv' 'warriors.csv']
No of Type : ['Comment' 'Reply']


In [10]:
trained_clf = 'doc2vec_rf.model'
loaded_clf = joblib.load(trained_clf)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
comments['SENTIMENT_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [12]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SENTIMENT_RF
0,Give Him Some Water Man,2018-12-03T10:11:17.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[give, him, some, water, man]","[-0.084303185, 0.43828505, 0.24113432, -0.1528...",1
1,What s going on Eminem,2018-12-03T10:09:18.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[what, s, going, on, eminem]","[0.14959222, -0.18436295, 0.3981964, 0.1262267...",0
2,No one can beat you,2018-12-03T09:46:24.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[no, one, can, beat, you]","[-0.06286393, 0.46472645, 0.2018007, 0.1916378...",0
3,EMINEM THE ALLTIME BEST RAPPER IN THE WORLD,2018-12-03T09:45:16.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[eminem, the, alltime, best, rapper, in, the, ...","[0.070282124, 0.07256028, 0.07732015, 0.011419...",1
4,Much love from india,2018-12-03T09:03:50.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[much, love, from, india]","[0.16853374, 0.22485605, 0.123892896, 0.045539...",1


In [13]:
trained_clf = 'doc2vec_kn.model'
loaded_clf = joblib.load(trained_clf)

In [14]:
comments['SENTIMENT_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [15]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SENTIMENT_RF,SENTIMENT_KN
0,Give Him Some Water Man,2018-12-03T10:11:17.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[give, him, some, water, man]","[-0.084303185, 0.43828505, 0.24113432, -0.1528...",1,1
1,What s going on Eminem,2018-12-03T10:09:18.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[what, s, going, on, eminem]","[0.14959222, -0.18436295, 0.3981964, 0.1262267...",0,0
2,No one can beat you,2018-12-03T09:46:24.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[no, one, can, beat, you]","[-0.06286393, 0.46472645, 0.2018007, 0.1916378...",0,1
3,EMINEM THE ALLTIME BEST RAPPER IN THE WORLD,2018-12-03T09:45:16.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[eminem, the, alltime, best, rapper, in, the, ...","[0.070282124, 0.07256028, 0.07732015, 0.011419...",1,1
4,Much love from india,2018-12-03T09:03:50.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,"[much, love, from, india]","[0.16853374, 0.22485605, 0.123892896, 0.045539...",1,1


In [16]:
out_df = comments[['Comment','CreateTimeStamp','Type', 'videoID', 'videoTitle', 'FNAME','SENTIMENT_RF','SENTIMENT_KN']]

In [17]:
out_df.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,SENTIMENT_RF,SENTIMENT_KN
0,Give Him Some Water Man,2018-12-03T10:11:17.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,1,1
1,What s going on Eminem,2018-12-03T10:09:18.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,0,0
2,No one can beat you,2018-12-03T09:46:24.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,0,1
3,EMINEM THE ALLTIME BEST RAPPER IN THE WORLD,2018-12-03T09:45:16.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,1,1
4,Much love from india,2018-12-03T09:03:50.000Z,Comment,j5-yKhDd64s,Eminem - Not Afraid,eminem-not-affraid.csv,1,1


In [18]:
out_file_name = '../outdata/video_comments_analysis_sentiments.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [19]:
print('****End of script****')

****End of script****
